# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Student 1 - __[antoine.bedaton@ulb.be](mailto:antoine.bedaton@ulb.be) - Student ID YYYYYY__
### Student 2 - __[pierre.defraene@ulb.be](mailto:pierre.defraene@ulb.be) - Student ID XXXXXX__
### Student 3 - __[nathan.marotte@ulb.be](mailto:nathan.marotte@ulb.be) - Student ID 459274__

### Video presentation: www.youtube.com/abcd1234

## Project Title


# Introduction


# Data preprocessing

## Missing value imputation

## Feature engineering

## Feature selection



In [29]:
# Read the dataset to the variable training_set_labels
training_set_labels <- read.csv(file = 'data/training_set_labels.csv')


# Adds en empty column functional_need_repairs
training_set_labels <- cbind(training_set_labels, "non_functional" = 0)
# Adds en empty column functional_need_repairs
training_set_labels <- cbind(training_set_labels, "functional_need_repairs" = 0)
# Adds an empty column functional
training_set_labels <- cbind(training_set_labels, "functional" = 0)

# Put 1 in the column depending on the status_group
training_set_labels$non_functional <- ifelse(training_set_labels$status_group == "non functional", 1, 0)
training_set_labels$functional_need_repairs <- ifelse(training_set_labels$status_group == "functional need repairs", 1, 0)
training_set_labels$functional <- ifelse(training_set_labels$status_group == "functional", 1, 0)
head(training_set_labels)

,id,status_group,non_functional,functional_need_repairs,functional
,<int>,<chr>,<dbl>,<dbl>,<dbl>
1,69572,functional,0,0,1
2,8776,functional,0,0,1
3,34310,functional,0,0,1
4,67743,non functional,1,0,0
5,19728,functional,0,0,1
6,9944,functional,0,0,1


# Model selection

## Model 1

## Model 2

## Model 3

#### Example of simple equation
\begin{equation}
e = mc^2
\end{equation}

#### Example of matrix equation - Cross product formula:

\begin{equation*}
\mathbf{V}_1 \times \mathbf{V}_2 =  \begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
\frac{\partial X}{\partial u} &  \frac{\partial Y}{\partial u} & 0 \\
\frac{\partial X}{\partial v} &  \frac{\partial Y}{\partial v} & 0
\end{vmatrix}
\end{equation*}

#### Example of multiline equation - The Lorenz Equations:

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

#### Example of Markdown Table:

| This | is   |
|------|------|
|   a  | table|


# Alternative models





# Conclusions